In [1]:
!pip3 install --upgrade numpy

In [2]:
# Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/kennong/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Price Volume data retrieval using Yahoo Finance API

Target tickers:
- S&P 500 Index (SPY)
- VIX Index (^VIX)
- Gold (GC=F)
- Brent Crude (BZ=F)
- WTI (CL=F)

In [3]:
# !apt-get install -y build-essential
# !apt-get install -y wget
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib/
# !./configure --prefix=/usr
# !make
# !make install
!pip uninstall -y numpy
!pip install numpy == 1.20.3
!pip install TA-Lib

import yfinance as yf
import talib

Found existing installation: numpy 1.26.1
Uninstalling numpy-1.26.1:
  Successfully uninstalled numpy-1.26.1
zsh:1: = not found
  Using cached numpy-1.26.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.1-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 1.26.1 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.1 which is incompatible.


In [4]:
# Create price extractor

class price_volume_data:
    """A class for retrieving and analyzing price and volume data."""
    def __init__(self, tickers, start_date, end_date):
        """
        Initializes a new instance of the price_volume_data class.

        Args:
            tickers (list): A list of tickers for the assets to retrieve data for.
            start_date (str): The start date for the data retrieval (format: 'YYYY-MM-DD').
            end_date (str): The end date for the data retrieval (format: 'YYYY-MM-DD').
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.pv = self.__retrieve_data__()
        
        #self.lp, self.ret = self.__get_daily_returns__()
        
    def __retrieve_data__(self):
        """
        Retrieves price and volume data for the given tickers from Yahoo Finance API.

        Returns:
            dict: A dictionary containing the price and volume data for each ticker.
        """

        data, failed = {}, []
        #shares_outstanding = []
        for ticker in self.tickers:
            try:
                df = yf.Ticker(ticker).history(start = self.start_date, end = self.end_date)
                #shares_outstanding.append(yf.Ticker(ticker).info['sharesOutstanding'])   
                
                data[ticker] = df
                
            except:
                failed.append(ticker)
                
        if failed:
            print('Tickers failed to extract: ', failed)
        
        return data
    
    def __get_daily_returns__(self):
        """
        Computes daily returns for the retrieved price data.

        Returns:
            pd.DataFrame: A DataFrame containing the daily price data.
            pd.DataFrame: A DataFrame containing the daily price returns.
        """
        df = pd.DataFrame()
        
        for ticker in self.pv:
            if df.empty:
                df = pd.DataFrame(self.pv[ticker]['Close'].copy())
            else:
                df = df.merge(self.pv[ticker]['Close'], how='outer',left_index=True, right_index=True)

        df.columns = self.tickers
        df = df.fillna(method = 'ffill')
        
        return df, df/df.shift(1) - 1
    
    def get_info(self, ticker = None):
        """
        Prints information about the price and volume data for the specified ticker.
        If no ticker is provided, prints information for all tickers.

        Args:
            ticker (str, optional): The ticker symbol for the asset. Defaults to None.
        """
        if ticker:
            self.pv[ticker].info()
        
        else:
            for ticker in self.pv:
                self.pv[ticker].info()
                
        return None
    
    def plot_price_trends(self, normalize = True, reference = -1):
        """
        Plots the close prices of selected assets over time.

        Args:
            normalize (bool, optional): Indicates whether to normalize the prices. Defaults to True.
            reference (int, optional): Reference point for normalization. Defaults to -1.
        """
        f, ax = plt.subplots(1,1,figsize=(12,5))
        # plot prices
        for key in self.pv:
            last_price = self.pv[key].loc[:, 'Close'].copy()
            if normalize:
                if reference == -1:
                    last_price /= last_price.iloc[0]
                elif reference == 0:
                    midpt = int(last_price.shape[0]/2)
                    last_price /= last_price.iloc[midpt]
                else:
                    last_price /= last_price.iloc[-1]
            last_price.plot(ax = ax, label = key)
        
        ax.title.set_text('Close Prices of selected assets')
        ax.set(ylabel = 'Normalized Price to start of observation')
        ax.legend()
        plt.show()
        
    def plot_corr(self, method='pearson', output = False, series = 'price'):
        """
        Plots the correlation matrix between asset prices or returns.

        Args:
            method (str, optional): The correlation method to use. Defaults to 'pearson'.
            output (bool, optional): Indicates whether to return the correlation matrix. Defaults to False.
            series (str, optional): Indicates whether to use 'price' or 'returns'. Defaults to 'price'.

        Returns:
            pd.DataFrame or None: The correlation matrix if output=True, None otherwise.
        """
        corr_matrix = self.lp.corr(method) if series == 'price' else self.ret.corr(method)
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        
        f, ax = plt.subplots(figsize=(5, 5))
        sns.heatmap(corr_matrix, mask=mask, vmin=-1, vmax=1, annot=True)
        
        return corr_matrix if output else None
        
        
    def get_technical_indicators(self, indicators = None, normalize = True):
        """
        Computes and adds various technical indicators to the price data.

        Args:
            indicators (list, optional): A list of tuples specifying the indicators and their parameters.
                Defaults to None.
            normalize (bool, optional): Indicates whether to normalize the indicator values. Defaults to True.
        """
        if not indicators:
            indicators = [('SMA', 5), ('SMA', 14), ('ATR', 5), ('ATR', 14), ('ADX', 5),
                         ('ADX', 14), ('RSI', 5), ('RSI', 14), ('MACD', None), ('BBANDS', None)]
        
        for data in self.pv:
            df = self.pv[data]
            for indicator, param in indicators:
                
                
                if indicator == 'SMA':
                    df[f'{indicator}_{param}'] = talib.SMA(df['Close'], param)

                elif indicator == 'ATR':
                    df[f'{indicator}_{param}'] = talib.ATR(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'ADX':
                    df[f'{indicator}_{param}'] = talib.ADX(df['High'], df['Low'], df['Close'], param) 
                    
                elif indicator == 'RSI':
                    df[f'{indicator}_{param}'] = talib.RSI(df['Close'], param) 
                    
                elif indicator == 'MACD':
                    df[f'{indicator}'], df[f'{indicator}_signal'], _ = talib.MACD(df['Close']) 
                    
                    
                elif indicator == 'BBANDS':
                    df[f'{indicator}_upp'], df[f'{indicator}_mid'], df[f'{indicator}_low'] = talib.BBANDS(df['Close'])
                    
                    
                else:
                    print('Indicators not built into the class. Please run TALIB manually!')
            
            df = df.replace([np.inf, -np.inf], np.nan, inplace = True)
            
        return None              

In [5]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM
from keras.regularizers import L1, L1L2, L2
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

import numpy as np
import tensorflow as tf
import random as python_random

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os

def reset_seeds(SEED = 624):
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

tf.config.experimental.enable_op_determinism()

2023-10-28 13:05:27.966458: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
class LSTM_predictor:
    """A class for predicting time series using LSTM (Long Short-Term Memory) models."""
    def __init__(self, X, Y):
        """
        Initializes a new instance of the LSTM_predictor class.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.
        """
        self.X, self.Y = self.__clean_data__(X, Y)
        self.features_size = X.shape[1]
        self.obs_size = X.shape[0]
        
        
    def __clean_data__(self, X, Y):
        """
        Cleans the input feature matrix and target values by removing rows with NaN values.

        Args:
            X (ndarray): The input feature matrix.
            Y (ndarray): The target values.

        Returns:
            ndarray: The cleaned input feature matrix.
            ndarray: The cleaned target values.
        """
        Z = np.concatenate((X, Y), axis = 1)
        Z = Z[~np.isnan(Z).any(axis=1), :]
        return Z[:, :-1], Z[:, -1]
        
    def prepare_data(self, train_test_split = 0.8, timestep = 1, look_forward = 1, inplace = False, scaled = True):
        """
        Prepares the data for training and testing the LSTM model.

        Args:
            train_test_split (float, optional): The fraction of data to be used for training.
                Defaults to 0.8.
            timestep (int, optional): The number of time steps to consider for each sample.
                Defaults to 3.
            inplace (bool, optional): Indicates whether to store the prepared data in the class
                instance or return it. Defaults to False.
            scaled (bool, optional): Indicates whether to scale the data. Defaults to True.

        Returns:
            tuple or None: If inplace=True, returns None. Otherwise, returns a tuple containing
            the prepared training and testing data as (X_train, Y_train, X_test, Y_test).
        """
        split_idx = int(self.obs_size*0.8)
        train_set = self.X[:-4-look_forward,:], self.Y[timestep+look_forward:-4]
        test_set = self.X[-4-timestep+1:,:], self.Y[-4+look_forward:]
        print(test_set[0])
        
        # Scale factors
        mean_X = np.mean(train_set[0], axis = 0)
        std_X = np.std(train_set[0], axis = 0)
        self.__scale_X__ = (mean_X, std_X)
        
        mean_Y = np.mean(train_set[1], axis = 0)
        std_Y = np.std(train_set[1], axis = 0)
        self.__scale_Y__ = (mean_Y, std_Y)
        
        sampled = []

        for X, Y in [train_set, test_set]:
            X_sampled, Y_sampled = [], []
            #print(X.shape)
            #print(Y.shape)
            print(len(X))

            for i in range(len(X)-timestep-look_forward+1):
                print(i)

                X_sampled.append((X[i:i+timestep] - self.__scale_X__[0])/self.__scale_X__[1])
                Y_sampled.append((Y[i] - self.__scale_Y__[0])/self.__scale_Y__[1])
            
            sampled.append((np.array(X_sampled),np.array(Y_sampled)))
        
        if inplace:
            self.X_train, self.Y_train = sampled[0]
            self.X_test, self.Y_test = sampled[1]
            print("x_train after prepare_data has shape of {}".format(self.X_train.shape))
            print("x_test after prepare_data has shape of {}".format(self.X_test.shape))
            
            return None
        else:
            return sampled
        
    def init_model(self, optimizer='adam', loss='mean_squared_error', kernel_regularizer=None, num_lstm_units = 64, num_lstm_units_2 = 64,num_dense_units = 25, learning_rate=0.001):
        """
        Initializes the LSTM model with the specified optimizer and loss function.

        Args:
            optimizer (str, optional): The optimizer to use. Defaults to 'adam'.
            loss (str, optional): The loss function to use. Defaults to 'mean_squared_error'.
        """
        reset_seeds()
        model = Sequential()
        model.add(LSTM(num_lstm_units, return_sequences=True,\
                       input_shape= (self.X_train.shape[1], self.features_size),\
                      kernel_regularizer=kernel_regularizer, activation = 'relu')) #added activation = "relu"
        model.add(LSTM(num_lstm_units_2, return_sequences=False, kernel_regularizer=kernel_regularizer, activation = 'relu'))
        model.add(Dense(num_dense_units))
        model.add(Dense(1))
        
        
        #opt = Adam(learning_rate=learning_rate) #to remove and default to the code directly below
        
        #if optimizer == 'adam':
            #opt = Adam(learning_rate=learning_rate)
        #elif optimizer == 'SGD':
            #opt = SGD(learning_rate=learning_rate)
        #else:
            #raise ValueError(f"Unsupported optimizer: {optimizer}")
        
        model.compile(optimizer = optimizer, loss = loss) # changed optimizer = optimizer to optimizer = opt
        self.model = model
        
        return None
    
    def fit_model_train(self, batch_size=100, epochs=50, callbacks=None):
        """
        Trains the LSTM model.

        Args:
            batch_size (int, optional): The batch size for training. 
            epochs (int, optional): The number of epochs to train for. 
        """
        self.test_loss_list = []
        self.pred_per_epoch = []
        history_train = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, verbose=1, callbacks = callbacks)
        self.train_loss_list = history_train.history['loss']  # Store loss values in the list 
       
        
    def predict(self):
        """Makes predictions using the trained LSTM model."""
        self.Y_pred = self.model.predict(self.X_test)
        #mse = mean_squared_error(self.Y_test, self.Y_pred)
        #mse_list.append(mse)
        

        
    def key_metrics(self,epoch=None):
        Y_actual = self.Y
        Y_pred = self.Y_pred if epoch == None else self.pred_per_epoch[epoch-1]
        mse = mean_squared_error(self.Y_test, Y_pred)
        mae = mean_absolute_error(self.Y_test, Y_pred)
        exp_ratio = explained_variance_score(self.Y_test, Y_pred)
        
        return Y_actual[-1],(Y_pred[-1]*self.__scale_Y__[1] + self.__scale_Y__[0]), mse, mae, exp_ratio 
    
    def cross_validation(self, n_splits = 5, batch_size=100, epochs=10, callback=False):
        """
        Perform cross-validation for the LSTM model.

        Parameters:
        - n_splits: Number of cross-validation splits (int, default=5)
        - batch_size: Number of samples per gradient update (int, default=100)
        - epochs: Number of epochs to train the model (int, default=50)
        - callback: Flag indicating whether to use custom callback during training (bool, default=True)

        Returns:
        - None
        """
        reset_seeds()
        tscv = TimeSeriesSplit(n_splits=5)
        
        cv_train_scores = []
        cv_val_scores = []
        
        print('Initiate cross-validation')
        i = 1

        for train_index, test_index in tscv.split(self.X_train):
            print(f'Running cross validation round {i}')
            self.temp_val_scores = []

            #print(self.X_train[train_index].shape)
            #print(self.X_train[test_index].shape)
            
            cv_X_train, cv_X_test = self.X_train[train_index], self.X_train[test_index]
            cv_y_train, cv_y_test = self.Y_train[train_index], self.Y_train[test_index]
            
            #print(cv_X_test.shape)
    
            # Fit your model on the training data
            history_train = self.model.fit(cv_X_train, cv_y_train, batch_size=batch_size,\
                                           epochs=epochs,\
                                           verbose = 2)
            cv_train_scores.append(history_train.history['loss'])
            cv_val_scores.append(self.temp_val_scores)
            print('==============================================')
            i+=1
            
        self.cv_train_scores = np.array(cv_train_scores).mean(axis = 0)
        self.cv_val_scores = np.array(cv_val_scores).mean(axis = 0)



In [7]:
sheet_name = ['20220526','20220825','20221124','20230223','20230525']
symbols = pd.read_excel("target_stock_universe.xlsx", sheet_name = sheet_name[-2]) # change here to get the other sheet
symbols['Ticker_Symbol'] = symbols['Ticker'].str.split().str[0]+".L"
symbols = symbols[symbols['Ticker_Symbol']!='HSV.L']
symbols = symbols[symbols['Ticker_Symbol']!='HBR.L']
symbols = symbols[symbols['Ticker_Symbol']!='TW/.L']
symbols = symbols[symbols['Ticker_Symbol']!='MDC.L']
symbols = symbols[symbols['Ticker_Symbol']!='HL/.L']

symbols_short = symbols['Ticker_Symbol'] # change here to get the full list of stock
data = price_volume_data(symbols_short.to_list(), '2018-02-24', '2023-02-28')
data.get_technical_indicators()
technical_indicators = ['Close', 'Volume', 'SMA_5', 'SMA_14', 'ATR_5', 'ATR_14', 'ADX_5', 'ADX_14', 'RSI_5','RSI_14', 'MACD', 'MACD_signal', 'BBANDS_upp', 'BBANDS_mid', 'BBANDS_low']

In [8]:
def get_technicals(technical_indicators, symbols_short, data):
    results = {}
    Y = {}
    X = {}
    for x in range(len(symbols_short)):
        results[x] = data.pv[f'{symbols_short.iloc[x]}']
        Y[x] = np.array(results[x]['Close']).reshape(-1,1)
        X[x] = np.array(results[x][technical_indicators])
    return results, Y, X

In [9]:
def run_LSTM(X,Y):
    results = {}
    for x in range(len(symbols_short)):
        model1 = LSTM_predictor(X[x], Y[x])
        model1.prepare_data(train_test_split = 0.8, timestep = 3, look_forward = 3, inplace = True)
        model1.init_model(optimizer='adam', loss='mean_squared_error')
        model1.cross_validation()
        model1.fit_model_train(epochs=30,batch_size=32)
        #return model1
        #model1.fit_model_train(epochs=30,batch_size=32,callbacks=[CustomCallback(model1)])
        model1.predict()
        #return model1
        model1.key_metrics()
        #model1.plot_prediction()
        results[x] = model1.key_metrics()
    return results
    

In [10]:
symbols_short

0     BDEV.L
1      EDV.L
3     ABDN.L
4     BLND.L
6     JMAT.L
7      HSX.L
8      UTG.L
9     FRAS.L
10     CCL.L
11    INVP.L
12     ICP.L
13     IMI.L
14    HWDN.L
15     HIK.L
16    INCH.L
18     EZJ.L
19     DPH.L
20     ITV.L
21    DPLM.L
22     IGG.L
23     SXS.L
24    TATE.L
25     EMG.L
26     TUI.L
27     GAW.L
28     MKS.L
29     DLN.L
30     RSW.L
31    BBOX.L
32     ROR.L
33     GRG.L
34     VTY.L
Name: Ticker_Symbol, dtype: object

In [11]:
results, Y, X = get_technicals(technical_indicators, symbols_short, data)

In [12]:
metrics = run_LSTM(X,Y)

[[4.33279755e+02 2.95077400e+06 4.31738605e+02 4.34002555e+02
  9.15882257e+00 9.91295241e+00 3.07495839e+01 3.22684184e+01
  5.29742877e+01 5.61174875e+01 6.41317099e+00 8.32125014e+00
  4.37384703e+02 4.31738605e+02 4.26092506e+02]
 [4.21210693e+02 7.98026600e+06 4.30587408e+02 4.33339419e+02
  1.04390185e+01 1.03162988e+01 3.12896630e+01 3.07036542e+01
  3.10313050e+01 4.78188699e+01 5.09665539e+00 7.67633119e+00
  4.40437784e+02 4.30587408e+02 4.20737033e+02]
 [4.22881744e+02 3.62827900e+06 4.28247864e+02 4.31442849e+02
  9.98703792e+00 1.01636429e+01 3.36495584e+01 2.88742176e+01
  3.56449026e+01 4.89446402e+01 4.14041897e+00 6.96914875e+00
  4.38730640e+02 4.28247864e+02 4.17765088e+02]
 [4.24367188e+02 2.77099500e+06 4.26372510e+02 4.30408356e+02
  9.08512969e+00 9.82891816e+00 3.20920160e+01 2.74291579e+01
  4.00974576e+01 4.99777594e+01 3.46254352e+00 6.26782770e+00
  4.35521437e+02 4.26372510e+02 4.17223583e+02]
 [4.20282257e+02 6.09492800e+06 4.24404327e+02 4.29519751e+02
  

2023-10-28 13:05:41.189050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [204,3,15]
	 [[{{node Placeholder/_10}}]]
2023-10-28 13:05:41.189398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [204]
	 [[{{node Placeholder/_11}}]]


3/3 - 3s - loss: 0.9173 - 3s/epoch - 995ms/step
Epoch 2/10
3/3 - 0s - loss: 0.7721 - 21ms/epoch - 7ms/step
Epoch 3/10
3/3 - 0s - loss: 0.6266 - 28ms/epoch - 9ms/step
Epoch 4/10
3/3 - 0s - loss: 0.4734 - 20ms/epoch - 7ms/step
Epoch 5/10
3/3 - 0s - loss: 0.3143 - 26ms/epoch - 9ms/step
Epoch 6/10
3/3 - 0s - loss: 0.1693 - 21ms/epoch - 7ms/step
Epoch 7/10
3/3 - 0s - loss: 0.1236 - 19ms/epoch - 6ms/step
Epoch 8/10
3/3 - 0s - loss: 0.1582 - 24ms/epoch - 8ms/step
Epoch 9/10
3/3 - 0s - loss: 0.1263 - 23ms/epoch - 8ms/step
Epoch 10/10
3/3 - 0s - loss: 0.0702 - 26ms/epoch - 9ms/step
Running cross validation round 2
Epoch 1/10
5/5 - 0s - loss: 0.1610 - 24ms/epoch - 5ms/step
Epoch 2/10
5/5 - 0s - loss: 0.1387 - 29ms/epoch - 6ms/step
Epoch 3/10
5/5 - 0s - loss: 0.1209 - 32ms/epoch - 6ms/step
Epoch 4/10
5/5 - 0s - loss: 0.0984 - 65ms/epoch - 13ms/step
Epoch 5/10


2023-10-28 13:05:44.479219: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [407]
	 [[{{node Placeholder/_11}}]]
2023-10-28 13:05:44.480049: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [407]
	 [[{{node Placeholder/_11}}]]


5/5 - 0s - loss: 0.0814 - 31ms/epoch - 6ms/step
Epoch 6/10
5/5 - 0s - loss: 0.0660 - 31ms/epoch - 6ms/step
Epoch 7/10
5/5 - 0s - loss: 0.0525 - 30ms/epoch - 6ms/step
Epoch 8/10
5/5 - 0s - loss: 0.0417 - 30ms/epoch - 6ms/step
Epoch 9/10
5/5 - 0s - loss: 0.0371 - 29ms/epoch - 6ms/step
Epoch 10/10
5/5 - 0s - loss: 0.0339 - 31ms/epoch - 6ms/step
Running cross validation round 3
Epoch 1/10
7/7 - 0s - loss: 0.1812 - 36ms/epoch - 5ms/step
Epoch 2/10
7/7 - 0s - loss: 0.1244 - 36ms/epoch - 5ms/step
Epoch 3/10
7/7 - 0s - loss: 0.0760 - 39ms/epoch - 6ms/step
Epoch 4/10
7/7 - 0s - loss: 0.0654 - 36ms/epoch - 5ms/step
Epoch 5/10


...

11/11 - 0s - loss: 0.0369 - 54ms/epoch - 5ms/step
Epoch 5/10
11/11 - 0s - loss: 0.0350 - 52ms/epoch - 5ms/step
Epoch 6/10
11/11 - 0s - loss: 0.0411 - 53ms/epoch - 5ms/step
Epoch 7/10
11/11 - 0s - loss: 0.0363 - 54ms/epoch - 5ms/step
Epoch 8/10
11/11 - 0s - loss: 0.0335 - 53ms/epoch - 5ms/step
Epoch 9/10
11/11 - 0s - loss: 0.0315 - 51ms/epoch - 5ms/step
Epoch 10/10
11/11 - 0s - loss: 0.0315 - 52ms/epoch - 5ms/step
Epoch 1/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0451
Epoch 2/30
 1/39 [..............................] - ETA: 0s - loss: 0.0378

2023-10-28 13:10:44.992952: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1219,3,15]
	 [[{{node Placeholder/_10}}]]
2023-10-28 13:10:44.993397: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1219]
	 [[{{node Placeholder/_11}}]]


39/39 [==============================] - 0s 4ms/step - loss: 0.0410
Epoch 3/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0352
Epoch 4/30
39/39 [==============================] - 0s 3ms/step - loss: 0.0299
Epoch 5/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0289
Epoch 6/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0252
Epoch 7/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0246
Epoch 8/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0259
Epoch 9/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0272
Epoch 10/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0234
Epoch 11/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0217
Epoch 12/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0282
Epoch 13/30
39/39 [==============================] - 0s 4ms/step - loss: 0.0218
Epoch 14/30
39/39 [==============================] - 0s 4ms

2023-10-28 13:10:49.507457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1,3,15]
	 [[{{node Placeholder/_10}}]]
2023-10-28 13:10:49.507661: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1,3,15]
	 [[{{node Placeholder/_10}}]]


1/1 [==============================] - 0s 258ms/step


In [13]:
metrics_copy = pd.DataFrame(metrics)

In [14]:
metrics_copy.to_csv("prediction_result_3days_20230223.csv")

In [ ]:
metrics_copy

In [ ]:
#market share NEAR rank date

def extract_sp(stocks, start, end):
    def data(ticker):
        return(yf.Ticker(ticker).get_shares_full(start= start, end=end))
    FAANG_Stock = map(data, stocks)
    return(pd.concat(FAANG_Stock,
                     keys = stocks,
                     names = ["Company", "Date"]
                    )
          )


tickers = symbols_short

result_ = extract_sp(tickers,start="2015-05-23", end="2023-05-23")
result_df = pd.DataFrame(result_, columns = ['Market Share'])
market_share_df =\
(
    result_df
    .reset_index()
    .sort_values(by='Date', ascending = False)
    .drop_duplicates(subset='Company', keep='first')

)

market_share_df

In [ ]:
market_share_df["Market Share"]

In [ ]:
shares_outstanding = pd.DataFrame(data.shares)

In [ ]:
shares_outstanding = shares_outstanding.T

In [ ]:
shares_outstanding.columns = symbols_short.to_list()

In [ ]:
shares_outstanding

In [ ]:
combined = pd.concat([metrics_copy,shares_outstanding], axis=0, join='outer')

In [ ]:
combined = combined.T

In [ ]:
combined

In [ ]:
combined['final_marketcap'] = combined[0]*combined['Actual Price']

In [ ]:
combined = combined.rename(columns={0: 'Market Share'})

In [ ]:
combined

In [ ]:
sorted_df = combined.sort_values(by='final_marketcap')

In [ ]:
sorted_df

In [ ]:
sorted_df.to_csv("results_stocks_comparison_20230523.csv")

In [ ]:
#################